In [1]:
import pandas as pd
import sqlite3

In [6]:
query = """
SELECT COUNT(*)
FROM nnpes AS np
LEFT JOIN nucc_taxonomy AS nt
ON np.primary_taxonomy = nt.code
LEFT JOIN zip_cbsa AS zp
ON np.postal_code = zp.zip
WHERE entity_type_code IS NOT NULL
LIMIT 10000;
"""

with sqlite3.connect('../data/hop_teaming_database.sqlite') as db: 
    test_sqlite = pd.read_sql(query, db)

test_sqlite

,COUNT(*)
0,8179521


In [32]:
query = """
SELECT *
--DISTINCT provider_organization_name
FROM nnpes AS np
INNER JOIN nucc_taxonomy AS nt
ON np.primary_taxonomy = nt.code
INNER JOIN zip_cbsa AS zc
ON np.postal_code = zc.zip
WHERE entity_type_code = 2.0
    AND state = 'TN'
    AND cbsa = 34980
    AND res_ratio = 1
    AND classification LIKE '%Hospital'
    AND provider_organization_name NOT LIKE '%PAY%'
"""

# LIMIT 1000;

with sqlite3.connect('../data/hop_teaming_database.sqlite') as db: 
    hospitals_sqlite = pd.read_sql(query, db)

hospitals_sqlite

,npi,entity_type_code,provider_organization_name,provider_last_name,provider_first_name,provider_middle_name,provider_name_prefix,provider_name_suffix,provider_credential_text,provider_first_line_business_practice_location_address,...,display_name,section,zip,cbsa,usps_zip_pref_city,usps_zip_pref_state,res_ratio,bus_ratio,oth_ratio,tot_ratio
0,1861675647,2.0,RIVER REGION HOSPITAL,None,None,None,None,None,None,49 MUSIC SQ W,...,Psychiatric Hospital,Non-Individual,37203,34980,NASHVILLE,TN,1.0,1.0,1.0,1.0
1,1265445506,2.0,WILLIAMSON COUNTY HOSPITAL DISTRICT,None,None,None,None,None,None,4321 CAROTHERS PARKWAY,...,General Acute Care Hospital,Non-Individual,37067,34980,FRANKLIN,TN,1.0,1.0,1.0,1.0
2,1609970029,2.0,"BUCKS COUNTY ONCOPLASTIC INSTITUTE, LLC",None,None,None,None,None,None,511 UNION ST,...,General Acute Care Hospital,Non-Individual,37219,34980,NASHVILLE,TN,1.0,1.0,1.0,1.0
3,1902062425,2.0,TENNESSEE VALLEY HEALTHCARE SERVICES,None,None,None,None,None,None,1310 24TH AVE S # 11T,...,Military General Acute Care Hospital,Non-Individual,37212,34980,NASHVILLE,TN,1.0,1.0,1.0,1.0
4,1215083316,2.0,"CENTER FOR COMPREHENSIVE SERVICES, INC.",None,None,None,None,None,None,268 GOVERNOR HALL RD,...,Rehabilitation Hospital,Non-Individual,37031,34980,CASTALIAN SPRINGS,TN,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,1558672543,2.0,TROUSDALE MEDICAL CENTER LLC,None,None,None,None,None,None,500 CHURCH ST,...,Critical Access Hospital,Non-Individual,37074,34980,HARTSVILLE,TN,1.0,1.0,1.0,1.0
80,1184987505,2.0,"TRUSTPOINT HOSPITAL, LLC",None,None,None,None,None,None,1009 N THOMPSON LN,...,General Acute Care Hospital,Non-Individual,37129,34980,MURFREESBORO,TN,1.0,1.0,1.0,1.0
81,1497012959,2.0,"TRUSTPOINT HOSPITAL, LLC",None,None,None,None,None,None,1009 N THOMPSON LN,...,Psychiatric Hospital,Non-Individual,37129,34980,MURFREESBORO,TN,1.0,1.0,1.0,1.0
82,1821350349,2.0,"TRUSTPOINT HOSPITAL, LLC",None,None,None,None,None,None,1009 N THOMPSON LN,...,Rehabilitation Hospital,Non-Individual,37129,34980,MURFREESBORO,TN,1.0,1.0,1.0,1.0
